In [138]:
from lxml import etree
import pickle
import nltk.data
import re
from nltk.tokenize import word_tokenize

In [140]:
from itertools import islice
def n_grams(a, n):
    z = (islice(a, i, None) for i in range(n))
    return zip(*z)

In [116]:
sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')

In [2]:
xml = open('1.1.text.xml', encoding='utf8').read()

In [3]:
tree = etree.fromstring(xml)

In [99]:
texts = tree.getchildren()

In [107]:
articles = []
for text in texts:
    title, abstract = text.getchildren()
    
    title = title.text
    ab_text = abstract.itertext()
    abstract = list(abstract)
    sequence = []
    for ab in ab_text:
        for el in abstract:
            if el.text == ab:
                sequence.append((ab, el.get('id')))
                _ = abstract.pop(abstract.index(el))
                break
        else:
            sequence.append((ab, 'O'))
    articles.append((title, sequence))

In [110]:
pickle.dump(articles, open('train_articles.pkl', 'wb'))

In [121]:
# articles[0]

In [128]:
sent_detector.tokenize(s)

[' Oral communication_H01-1001.1 is ubiquitous and carries important information yet it is also time consuming to document.',
 'Given the development of storage media and networks_H01-1001.2 one could just record and store a conversation_H01-1001.3 for documentation.',
 'The question is, however, how an interesting information piece would be found in a large database_H01-1001.4 .',
 'Traditional information retrieval techniques_H01-1001.5 use a histogram_H01-1001.6 of keywords_H01-1001.7 as the document representation_H01-1001.8 but oral communication_H01-1001.9 may offer additional indices_H01-1001.10 such as the time and place of the rejoinder and the attendance.',
 'An alternative index_H01-1001.11 could be the activity such as discussing, planning, informing, story-telling, etc.',
 'This paper addresses the problem of the automatic detection_H01-1001.12 of those activities in meeting situation and everyday rejoinders.',
 'Several extensions of this basic idea are being discussed an

In [127]:
s = ''.join([text[0]+(('_'+text[1]) if text[1] != 'O' else '') for text in articles[0][1]])


In [165]:
texts = []
for article in articles:
    text = []
    for t in article[1]:
        if t[1] == 'O':
            text.append(t[0])
        else:
            text.append(t[0].replace(' ', '_') + '_' + t[1])
    text = ''.join(text)
    sents = sent_detector.tokenize(text)
    texts.append((article[0], sents))

In [203]:
relations = []
for article in texts:
    
    for sent in article[1]:
        entities = re.findall('[\w\.\-\(\)\[\]\'\/\,\=\#]+(?:_[\w\.\-\(\)\[\]\'\/\,\=\#]+)+', sent)
        rel = []
        if len(entities) > 1:
            words = sent.split(' ')
            ws = []
            for word in words:
                w = re.split('(\W+)', word)
                ws += w
            words = ws
            for i in range(len(entities)):
                for j in range(1, len(entities[i+1:])):
                    start = words.index(entities[i]) + 1
                    end = words.index(entities[i+j])

                    context = words[start:end]
                    clear_context = []
                    for word in context:
                        if '_' in word:
                            w = word.split('_')[:-1]
                            clear_context += w
                        else:
                            clear_context += [word]
                    rel.append((entities[i], clear_context, entities[i+j]))
        relations += rel

ValueError: 'information_retrieval_techniques_H01-1001.5' is not in list

In [206]:
words

['Traditional',
 'information_retrieval_techniques_H01',
 '-',
 '1001',
 '.',
 '5',
 'use',
 'a',
 'histogram_H01',
 '-',
 '1001',
 '.',
 '6',
 'of',
 'keywords_H01',
 '-',
 '1001',
 '.',
 '7',
 'as',
 'the',
 'document_representation_H01',
 '-',
 '1001',
 '.',
 '8',
 'but',
 'oral_communication_H01',
 '-',
 '1001',
 '.',
 '9',
 'may',
 'offer',
 'additional',
 'indices_H01',
 '-',
 '1001',
 '.',
 '10',
 'such',
 'as',
 'the',
 'time',
 'and',
 'place',
 'of',
 'the',
 'rejoinder',
 'and',
 'the',
 'attendance',
 '.',
 '']

In [158]:
gold_relations = [re.findall('([A-Z]+)\(([\w\.\-]+)\,([\w\.\-]+)(?:\,([\w\.\-]+))?\)', text) 
                  for text in open('1.1.relations.txt', encoding='utf8').read().splitlines()]


In [163]:
rel_dict = {}
for rel in gold_relations:
    rel = rel[0]
    rel_dict[(rel[1], rel[2])] = (rel[0], rel[3])


In [164]:
rel_dict

{('H05-1095.11', 'H05-1095.12'): ('USAGE', 'REVERSE'),
 ('J90-3002.8', 'J90-3002.9'): ('USAGE', ''),
 ('C04-1011.6', 'C04-1011.7'): ('WHOLE', 'REVERSE'),
 ('A00-2023.8', 'A00-2023.9'): ('COMPARE', ''),
 ('A00-2023.2', 'A00-2023.3'): ('FEATURE', ''),
 ('P05-1067.4', 'P05-1067.5'): ('USAGE', 'REVERSE'),
 ('N03-2036.1', 'N03-2036.2'): ('USAGE', ''),
 ('P85-1015.7', 'P85-1015.9'): ('USAGE', ''),
 ('P84-1064.6', 'P84-1064.7'): ('WHOLE', 'REVERSE'),
 ('H01-1042.1', 'H01-1042.3'): ('USAGE', ''),
 ('C92-3165.16', 'C92-3165.17'): ('WHOLE', ''),
 ('H92-1003.1', 'H92-1003.2'): ('FEATURE', 'REVERSE'),
 ('L08-1110.5', 'L08-1110.6'): ('USAGE', 'REVERSE'),
 ('P06-4011.2', 'P06-4011.3'): ('WHOLE', ''),
 ('P97-1040.1', 'P97-1040.2'): ('FEATURE', ''),
 ('C96-1055.1', 'C96-1055.2'): ('WHOLE', ''),
 ('E89-1040.5', 'E89-1040.6'): ('WHOLE', ''),
 ('P82-1035.23', 'P82-1035.24'): ('WHOLE', ''),
 ('P81-1032.13', 'P81-1032.14'): ('USAGE', 'REVERSE'),
 ('C90-3046.5', 'C90-3046.6'): ('FEATURE', ''),
 ('C90-1002.1

In [137]:
re.findall('\w+(?:_[\w\.\-]+)+', sents[1])

['storage_media_and_networks_H01-1001.2', 'conversation_H01-1001.3']

In [101]:
t.getchildren()[1].getchildren()

[<Element entity at 0x1709488>,
 <Element entity at 0x1711408>,
 <Element entity at 0x1711448>,
 <Element entity at 0x17115c8>,
 <Element entity at 0x1711608>,
 <Element entity at 0x1711648>,
 <Element entity at 0x1711688>,
 <Element entity at 0x17116c8>,
 <Element entity at 0x1711708>,
 <Element entity at 0x1711748>,
 <Element entity at 0x1711788>,
 <Element entity at 0x17117c8>,
 <Element entity at 0x1711808>,
 <Element entity at 0x1711848>,
 <Element entity at 0x1711888>,
 <Element entity at 0x17118c8>,
 <Element entity at 0x1711908>,
 <Element entity at 0x1711948>,
 <Element entity at 0x1711988>,
 <Element entity at 0x17119c8>,
 <Element entity at 0x1711a08>]

In [98]:
t.getchildren()

[]

In [102]:
abstract = t.getchildren()[1]

In [105]:
list(abstract)

[<Element entity at 0x1709488>,
 <Element entity at 0x1711408>,
 <Element entity at 0x1711448>,
 <Element entity at 0x17115c8>,
 <Element entity at 0x1711608>,
 <Element entity at 0x1711648>,
 <Element entity at 0x1711688>,
 <Element entity at 0x17116c8>,
 <Element entity at 0x1711708>,
 <Element entity at 0x1711748>,
 <Element entity at 0x1711788>,
 <Element entity at 0x17117c8>,
 <Element entity at 0x1711808>,
 <Element entity at 0x1711848>,
 <Element entity at 0x1711888>,
 <Element entity at 0x17118c8>,
 <Element entity at 0x1711908>,
 <Element entity at 0x1711948>,
 <Element entity at 0x1711988>,
 <Element entity at 0x17119c8>,
 <Element entity at 0x1711a08>]

In [44]:
?abstract.get

In [81]:
ab = list(abstract.itertext())

In [65]:

for el in abstract.itertext():
    print(el)

 
Oral communication
 is ubiquitous and carries important information yet it is also time consuming to document. Given the development of 
storage media and networks
 one could just record and store a 
conversation
 for documentation. The question is, however, how an interesting information piece would be found in a 
large database
 . Traditional 
information retrieval techniques
 use a 
histogram
 of 
keywords
 as the 
document representation
 but 
oral communication
 may offer additional 
indices
 such as the time and place of the rejoinder and the attendance. An alternative 
index
 could be the activity such as discussing, planning, informing, story-telling, etc. This paper addresses the problem of the 
automatic detection
 of those activities in meeting situation and everyday rejoinders. Several extensions of this basic idea are being discussed and/or evaluated: Similar to activities one can define subsets of larger 
database
 and detect those automatically which is shown on a larg

In [82]:
seq = []
for el in abstract.iterchildren():
    for a in ab:
        if el.text == a:
            seq.append((ab.pop(ab.index(a)), el))
            break
        else:
            seq.append((a, 'O'))

In [84]:
for el in abstract.iterchildren():
    print(el.text)

Oral communication
storage media and networks
conversation
large database
information retrieval techniques
histogram
keywords
document representation
oral communication
indices
index
automatic detection
database
database
TV shows
Emotions
indices
dominance distribution of speakers
surface
databases
indices


In [87]:
e = seq[1][1]

In [95]:
e.get('id')

'H01-1001.1'

In [64]:
stringify_children(tree)

TypeError: sequence item 2: expected str instance, bytes found

In [63]:
el

<Element entity at 0x1711a08>

In [19]:
import gensim
from nltk.corpus import stopwords
from collections import Counter
stops = stopwords.words('english')

In [18]:
words = [x.split() for x in open('sents_norm.txt').read().splitlines()]

In [22]:
ph = gensim.models.Phrases(p[words], scoring='npmi', threshold=0.3)

In [10]:
p = gensim.models.phrases.Phraser(ph)

In [23]:
p2 = gensim.models.phrases.Phraser(ph)

In [24]:
phrases = Counter()
for x in words:
    phr = p2[p[x]]
    phrases.update([x for x in phr if '_' in x])

In [25]:
phrases.most_common(100)

[('for_example', 3842),
 ('in_case', 3374),
 ('et_al', 3340),
 ('cambridge_cambridge_university_press', 2305),
 ('in_addition', 1878),
 ('oxford_oxford_university_press', 1635),
 ('noun_phrase', 1634),
 ('-PRON-_argue', 1604),
 ('-PRON-_think', 1587),
 ('on_hand', 1577),
 ('word_order', 1480),
 ('in_fact', 1460),
 ('indo_european', 1414),
 ('for_instance', 1412),
 ('direct_object', 1329),
 ('lexical_item', 1269),
 ('â_�', 1242),
 ('in_order', 1208),
 ('proto_indo_european', 1199),
 ('take_place', 1164),
 ('-PRON-_not', 1142),
 ('in_chapter', 1141),
 ('point_view', 1112),
 ('proto_nostratic', 1107),
 ('natural_language', 1073),
 ('speech_act', 994),
 ('speaker_’s', 978),
 ('relative_clause', 944),
 ('this_mean', 942),
 ('second_language', 922),
 ('proto_nostratic_root', 913),
 ('native_speaker', 875),
 ('-PRON-_want', 845),
 ('argument_structure', 828),
 ('-PRON-_be', 818),
 ('language_acquisition', 807),
 ('syntactic_structure', 802),
 ('of_course', 764),
 ('answer_question', 764),
 ('